### Introduction to Tools in LangChain

Large Language Models (LLMs) are powerful reasoning engines, but they are isolated from the outside world. They can't access real-time data or perform actions on their own. **Tools** bridge this gap.

Tools allow models to interact with the world, such as:
-   Fetching data from a database or API (e.g., weather, stock prices).
-   Searching the web.
-   Performing complex calculations.

**Core Concepts of a Tool:**
1.  **Schema**: A structured definition (JSON) that tells the model the tool's **name**, **description**, and **arguments** (inputs). The model uses this to "decide" if and how to call the tool.
2.  **Function**: The actual Python code that performs the logic (the "work").
3.  **Tool Object**: A wrapper in LangChain that combines the function and schema.

### 1. Defining a Tool with `@tool`

The `@tool` decorator is the easiest way to define a custom tool. It automatically extracts the schema from your Python function:
-   **Name**: Taken from the function name (e.g., `get_weather`).
-   **Description**: Taken from the function's docstring. **Crucial**: This tells the model *when* to use the tool.
-   **Arguments**: Taken from the function arguments and type hints (e.g., `location: str`).

In [6]:
import os
from langchain.chat_models import init_chat_model
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, ToolMessage
from dotenv import load_dotenv

load_dotenv()

# Define the tool with hardcoded logic
@tool
def get_weather(location: str):
    """Get the weather for a specific location."""
    location = location.lower()
    if "london" in location:
        return "It is rainy in London."
    elif "new york" in location:
        return "It is sunny in New York."
    elif "kolkata" in location:
        return "It is sunny in Kolkata."
    else:
        return f"The weather in {location} is partly cloudy."

### 2. Binding Tools to the Model

Start by initializing a chat model. Then, use `.bind_tools([tool_list])`.

**Why bind tools?**
Standard LLMs generate text. Models fine-tuned for tool calling (like `llama-3.1-8b`, `gpt-4`, etc.) need to know what tools are available. `bind_tools` converts your Python tool definitions into the specific JSON format the model provider expects (e.g., OpenAI function calling schema).

In [5]:
# Initialize model
model = init_chat_model("llama-3.1-8b-instant", model_provider="groq")

# Bind tool to model
model_with_tools = model.bind_tools([get_weather])

model_with_tools

### 3. The Agent Loop (Manual Implementation)

Modern Agents (like those in **LangGraph**) typically run a loop. Here, we implement a single iteration manually to understand what happens under the hood.

**The Workflow:**

1.  **Thinking (Model Generation)**: We send the user's query to the model. The model analyzes the query and the available tools.
2.  **Tool Call Generation**: Instead of a text response, the model returns a **Tool Call**. This contains:
    -   The name of the tool to call (`get_weather`).
    -   The arguments to pass (`{"location": "Kolkata"}`).
    -   A unique ID for the call.
3.  **Execution**: The application (us) detects the tool call. We invoke the actual Python function `get_weather` using the provided arguments.
4.  **Observation**: We get the output string ("It is sunny in Kolkata."). We must pass this back to the model as a `ToolMessage` linked to the original call ID.
5.  **Final Response**: The model receives the tool's output. It uses this information to generate a natural language response for the user.

In [2]:
print("--- Step 1: Model generates tool call ---")
# step1: Model generates tool call
message = [HumanMessage(content="What's the weather in Kolkata?")]
ai_msg = model_with_tools.invoke(message)
message.append(ai_msg)
print(f"AI Message Content: {ai_msg.content}")
if ai_msg.tool_calls:
    print(f"Tool Calls: {ai_msg.tool_calls}")

# step2: Iterate and execute and collect the results
if ai_msg.tool_calls:
    print("\n--- Step 2: Iterate and execute ---")
    for tool_call in ai_msg.tool_calls:
        # execute the tool with generated arguments
        # invoking tool_call directly works if it's a ToolCall dict? No, usually expects args.
        # But let's use the explicit args to be safe and clear.
        tool_output = get_weather.invoke(tool_call)
        print(f"Tool Output: {tool_output}")
        
        # Create a ToolMessage to strictly follow LangChain's message history format
        tool_message = ToolMessage(
            tool_call_id=tool_call["id"],
            content=str(tool_output),
            name=tool_call["name"]
        )
        message.append(tool_message)

# step3: Pass the result back to the model for final response
print("\n--- Step 3: Final Response ---")
final_response = model_with_tools.invoke(message)
# Use .content instead of .text
print(final_response.content)